In [1]:
import pandas as pd
import numpy as np
data=pd.read_excel("赛题1数据材料\赛题1数据集\融资信息.xlsx")
data

,企业编号,融资日期,轮次,投资金额
0,1001,1999-11-10,IPO,未披露
1,1002,2003-04-28,IPO,未披露
2,1002,2017-12-21,战略投资,未披露
3,1003,1999-07-27,IPO,未披露
4,1004,1999-03-12,IPO,未披露
5,1005,2000-04-27,IPO,未披露
6,1006,1998-02-18,IPO,未披露
7,1007,2001-03-09,IPO,未披露
8,1008,2001-12-06,上市,278250万
9,1008,2005-09-26,上市后,123000万


In [4]:
data["投资金额"].value_counts()#66%为未披露，不要了

未披露           3360
10000.0         16
数千万人民币          13
2000.0          12
3000.0          11
数百万人民币          11
4000.0           9
6000.0           9
5000.0           9
1500.0           9
3600.0           8
2500.0           8
1000.0           8
8000.0           8
28000.0          7
100.0            6
45000.0          6
50000.0          6
30000.0          6
数亿美元             6
1200.0           5
300000.0         5
12000.0          5
96000.0          5
40000.0          5
3300.0           5
7500.0           4
34000.0          4
9000.0           4
7000.0           4
              ... 
29300.0          1
2670.0           1
17720.0          1
81988.0          1
80080.0          1
138320.0         1
9829.0           1
38500.0          1
5720.0           1
155232.0         1
10000000.0       1
13200.0          1
20100.0          1
1193.0           1
996.0            1
1900000.0        1
95900.0          1
667.0            1
1690.0           1
8840.0           1
6040.0           1
516000.0    

In [5]:
data=data.set_index("轮次").loc[["IPO","战略投资","A轮","上市","主板定向增发",
                          "B轮","天使轮","天使轮","股权转让","被收购","C轮","新三板"],:].reset_index()
for i in range(len(data)):
    if (data.iloc[i,3]!="未披露" and type(data.iloc[i,3])==np.str 
        and data.iloc[i,3][0]!="数"):
        if(data.iloc[i,3][-1]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-1])
        elif(data.iloc[i,3][-2]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-2])
        elif(data.iloc[i,3][-3]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-3])
        elif(data.iloc[i,3][-4]=="万"):
             data.iloc[i,3]=float(data.iloc[i,3][:-4])

  